In [1]:
# 线性回归的简介实现
import torch
import numpy as np
import random
from torch.utils import data # 处理数据的一些模块
from d2l import torch as d2l

In [2]:
def synthetic_data(true_w, true_b, num_example):
    # 生成y=Xw + b的噪声
    X = torch.normal(0, 1, size=(num_example, len(true_w)))
    y = torch.matmul(X, true_w) + true_b
    y += torch.normal(0, 0.01, y.shape)
    return X, y # 对应features, labels


In [3]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2

# 生成数据
features, label = d2l.synthetic_data(true_w, true_b, 1000)

features, label

(tensor([[-0.9627,  1.0769],
         [-1.4025,  0.9783],
         [-1.3203,  0.1880],
         ...,
         [ 2.3254, -0.0503],
         [ 0.3138,  0.1835],
         [ 1.5478,  0.6698]]),
 tensor([[-1.3908e+00],
         [-1.9318e+00],
         [ 9.0234e-01],
         [ 1.2317e+01],
         [ 2.2873e+00],
         [ 1.8396e+00],
         [ 3.9703e+00],
         [ 5.7814e+00],
         [ 1.7958e+00],
         [ 1.0690e+01],
         [ 2.2759e+00],
         [ 7.5912e+00],
         [ 2.5235e+00],
         [ 2.1924e+00],
         [ 5.8050e-01],
         [ 2.5577e+00],
         [ 9.5586e+00],
         [ 3.0292e+00],
         [-1.4495e+00],
         [ 7.3944e+00],
         [ 4.2299e+00],
         [ 5.0111e+00],
         [-2.9069e+00],
         [ 6.4099e+00],
         [ 4.4283e+00],
         [ 7.9741e+00],
         [ 2.2406e-01],
         [ 6.4884e+00],
         [ 4.1695e+00],
         [ 4.2533e+00],
         [ 5.0887e+00],
         [ 4.0339e+00],
         [ 5.1522e+00],
         [ 6.4839e

In [4]:
# 构造数据DataLoader: 
def load_dataloader(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays) # 这里解包, 返回Dataset对象
    # dataset, 就是数据集, 包含features和lables的数据集合. 
    # construct DataLoader
    return data.DataLoader(dataset, batch_size, shuffle=is_train)
    # DataLoader就是将全部数据整理成batch大小的批量数据, 并且随机打乱.

In [5]:
# 设置超参数
batch_size = 10
data_iter = load_dataloader((features, label), batch_size)

type(data_iter) # is DataLoader type, but is a iterator essentially

torch.utils.data.dataloader.DataLoader

In [6]:
#data_iter is a iterator, so we can use 'next()' to iter it, notice, we need iter transformation it
next(iter(data_iter)) # use iter() to trans type

[tensor([[-1.4503,  1.1889],
         [ 0.1509,  1.3706],
         [-0.8714, -2.1999],
         [-0.4587,  0.9042],
         [ 0.3772,  1.2176],
         [ 0.0386,  0.1078],
         [ 1.1653, -0.2459],
         [ 1.0528, -0.6142],
         [ 1.2615, -0.2692],
         [ 2.0321, -0.5839]]),
 tensor([[-2.7398],
         [-0.1451],
         [ 9.9277],
         [ 0.1993],
         [ 0.8181],
         [ 3.9122],
         [ 7.3772],
         [ 8.3885],
         [ 7.6222],
         [10.2419]])]

In [7]:
# 定义模型：
from torch import nn # network 


net = nn.Sequential(nn.Linear(2, 1)) # Sequentia: 一个容器
net
# Sequential(
#   (0): Linear(in_features=2, out_features=1, bias=True)
# )
# 同样也可以进行索引.

Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
)

In [8]:
# 初始化模型参数:
# 线性层中有两个参数: 就是weight就是权重, bias就是参数.
# data就是数据. 同时原地操作进行赋值
dir(net[0]) # 查看该对象的属性. 
net[0].weight.data.normal_(0, 0.01) # 
net[0].bias.data.fill_(0)

tensor([0.])

In [9]:
dir(net[0])

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__constants__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_d

In [10]:
# 定义损失函数
loss = nn.MSELoss() # mean squre error均方误差


In [11]:
# 定义优化算法

"""
net.parameters()返回一个生成器, 这样迭代时速度更快. 
"""
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

In [12]:
# 训练:
num_epoch = 3
for epoch in range(num_epoch): # 
    for X, y in data_iter: # 遍历batch数据
        l = loss(net(X), y)
        trainer.zero_grad() # empty gradient
        l.backward() # 反向传播
        trainer.step() # 更新
    
    # 一轮结束, 模型评估
    l = loss(net(features), label)
    print(f"epoch: {epoch+1}, loss: {l.item()}")
        

epoch: 1, loss: 0.00018450798233971
epoch: 2, loss: 9.856696124188602e-05
epoch: 3, loss: 9.847475303104147e-05


In [13]:
w = net[0].weight.data
print('w的估计误差：', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('b的估计误差：', true_b - b)

# 可见, 差距非常的小. 足以满足我们的要求. 

w的估计误差： tensor([0.0004, 0.0002])
b的估计误差： tensor([-0.0004])
